In [134]:
#!pip install yfinance pandas numpy scikit-learn requests

from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.decomposition import PCA

FUND = "XLU"
END = datetime.today().date()
START = END - timedelta(days=280)         # ~9 calendar months to ensure >= ~6 months trading days
TARGET_TRADING_DAYS = 126                 # ~6 months of trading days

TICKERS = [
    "NEE",  # NEXTERA ENERGY INC
    "SO",   # SOUTHERN CO/THE
    "CEG",  # CONSTELLATION ENERGY
    "DUK",  # DUKE ENERGY CORP
    "VST",  # VISTRA CORP
    "AEP",  # AMERICAN ELECTRIC POWER
    "SRE",  # SEMPRA
    "D",    # DOMINION ENERGY INC
    "XEL",  # XCEL ENERGY INC
    "EXC",  # EXELON CORP
    "PEG",  # PUBLIC SERVICE ENTERPRISE GP
    "ETR",  # ENTERGY CORP
    "WEC",  # WEC ENERGY GROUP INC
    "ED",   # CONSOLIDATED EDISON INC
    "PCG",  # P G + E CORP
    "NRG",  # NRG ENERGY INC
    "DTE",  # DTE ENERGY COMPANY
    "AEE",  # AMEREN CORPORATION
    "PPL",  # PPL CORP
    "ATO",  # ATMOS ENERGY CORP
    "AWK",  # AMERICAN WATER WORKS CO INC
    "ES",   # EVERSOURCE ENERGY
    "CNP",  # CENTERPOINT ENERGY INC
    "FE",   # FIRSTENERGY CORP
    "CMS",  # CMS ENERGY CORP
    "EIX",  # EDISON INTERNATIONAL
    "NI",   # NISOURCE INC
    "EVRG", # EVERGY INC
    "LNT",  # ALLIANT ENERGY CORP
    "PNW",  # PINNACLE WEST CAPITAL
    "AES",  # AES CORP
]


In [135]:
symbols = [FUND] + TICKERS

# Pull a wider window (START..END) to be safe, then trim to the last TARGET_TRADING_DAYS
raw = yf.download(
    tickers=symbols,
    start=START,                             # from your code above
    end=(END + timedelta(days=1)),           # include END's session
    interval="1d",
    auto_adjust=True,
    progress=False,
    group_by="ticker",
)

# Robustly extract a prices matrix regardless of single/multi-index
if isinstance(raw.columns, pd.MultiIndex):
    prices = raw.xs("Close", axis=1, level=1)
else:
    # single ticker case
    col = "Close" if "Close" in raw.columns else next(c for c in raw.columns if c.lower()=="close")
    prices = raw[col].to_frame(symbols[0])

# Align, ffill, ensure full panel, then trim
prices = (
    prices.reindex(columns=[c for c in symbols if c in prices.columns])
          .dropna(axis=1, how="all")
          .ffill()
          .dropna(how="any")
)

if len(prices) < TARGET_TRADING_DAYS:
    raise ValueError(f"Need {TARGET_TRADING_DAYS} days but only have {len(prices)}; extend START.")

prices_6m = prices.tail(TARGET_TRADING_DAYS)
fund_prices = prices_6m[FUND]
component_prices = prices_6m.drop(columns=[FUND])

In [136]:
print("\n=== Fund prices (head) ===")
print(fund_prices.head())

print("\n=== Component prices (head) ===")
print (component_prices.head)


=== Fund prices (head) ===
Date
2025-03-28    76.924515
2025-03-31    77.782730
2025-04-01    77.980026
2025-04-02    78.345016
2025-04-03    77.851784
Name: XLU, dtype: float64

=== Component prices (head) ===
<bound method NDFrame.head of Ticker            NEE         SO         CEG         DUK         VST  \
Date                                                                   
2025-03-28  69.348259  89.658920  204.862808  117.319519  118.771446   
2025-03-31  69.781387  90.465675  201.112473  119.834702  117.165207   
2025-04-01  69.810905  90.337776  206.408844  118.950455  121.953979   
2025-04-02  69.377800  89.826180  213.909531  118.282356  126.892395   
2025-04-03  71.011833  91.065834  189.751694  121.878288  107.956802   
...               ...        ...         ...         ...         ...   
2025-09-22  72.349998  92.330002  347.119995  121.580002  217.919998   
2025-09-23  72.320000  93.720001  336.649994  123.099998  204.240005   
2025-09-24  73.830002  94.410004  339.

In [137]:
# Arithmetic returns
rets = component_prices.pct_change().dropna()

# Log returns
log_rets = np.log(component_prices / component_prices.shift(1)).dropna()

In [138]:
print(f"\n=== Daily returns shape: {rets.shape} ===")
print(rets.head())


=== Daily returns shape: (125, 31) ===
Ticker           NEE        SO       CEG       DUK       VST       AEP  \
Date                                                                     
2025-03-31  0.006246  0.008998 -0.018307  0.021439 -0.013524  0.021597   
2025-04-01  0.000423 -0.001414  0.026335 -0.007379  0.040872 -0.008236   
2025-04-02 -0.006204 -0.005663  0.036339 -0.005617  0.040494 -0.005721   
2025-04-03  0.023553  0.013801 -0.112935  0.030401 -0.149226  0.012622   
2025-04-04 -0.072498 -0.039110 -0.101346 -0.041274 -0.093707 -0.042434   

Ticker           SRE         D       XEL       EXC  ...        ES       CNP  \
Date                                                ...                       
2025-03-31  0.023376  0.019640  0.015347  0.030642  ...  0.016198 -0.000552   
2025-04-01  0.002803  0.003389 -0.000989 -0.003472  ... -0.000322  0.015733   
2025-04-02  0.017887 -0.000533  0.002545  0.001089  ...  0.008697  0.003533   
2025-04-03 -0.028967  0.000356  0.017489  0.02

In [139]:
print(f"\n=== Daily log returns shape: {log_rets.shape} ===")
print(log_rets.head())


=== Daily log returns shape: (125, 31) ===
Ticker           NEE        SO       CEG       DUK       VST       AEP  \
Date                                                                     
2025-03-31  0.006226  0.008958 -0.018476  0.021212 -0.013616  0.021367   
2025-04-01  0.000423 -0.001415  0.025995 -0.007406  0.040059 -0.008270   
2025-04-02 -0.006223 -0.005679  0.035694 -0.005632  0.039696 -0.005738   
2025-04-03  0.023280  0.013706 -0.119837  0.029948 -0.161608  0.012543   
2025-04-04 -0.075260 -0.039895 -0.106857 -0.042150 -0.098392 -0.043361   

Ticker           SRE         D       XEL       EXC  ...        ES       CNP  \
Date                                                ...                       
2025-03-31  0.023107  0.019450  0.015231  0.030182  ...  0.016068 -0.000552   
2025-04-01  0.002799  0.003383 -0.000989 -0.003478  ... -0.000322  0.015610   
2025-04-02  0.017729 -0.000533  0.002542  0.001088  ...  0.008660  0.003526   
2025-04-03 -0.029395  0.000356  0.017338  

In [140]:
cov = log_rets.cov()
print("\n=== Covariance matrix (head) ===")
print(cov.iloc[:10, :10])


=== Covariance matrix (head) ===
Ticker       NEE        SO       CEG       DUK       VST       AEP       SRE  \
Ticker                                                                         
NEE     0.000365  0.000094  0.000104  0.000094  0.000085  0.000102  0.000128   
SO      0.000094  0.000104  0.000023  0.000100 -0.000014  0.000101  0.000091   
CEG     0.000104  0.000023  0.001052  0.000006  0.001001  0.000049  0.000236   
DUK     0.000094  0.000100  0.000006  0.000122 -0.000046  0.000112  0.000082   
VST     0.000085 -0.000014  0.001001 -0.000046  0.001273  0.000010  0.000230   
AEP     0.000102  0.000101  0.000049  0.000112  0.000010  0.000135  0.000104   
SRE     0.000128  0.000091  0.000236  0.000082  0.000230  0.000104  0.000245   
D       0.000151  0.000111  0.000128  0.000109  0.000122  0.000112  0.000159   
XEL     0.000159  0.000107  0.000086  0.000115  0.000043  0.000121  0.000134   
EXC     0.000103  0.000102  0.000005  0.000116 -0.000042  0.000117  0.000097   

Ticke

In [141]:
rets_std = (log_rets - log_rets.mean()) / log_rets.std(ddof=1)
ncomp = min(10, rets_std.shape[1])
pca = PCA(n_components=ncomp).fit(rets_std)

evr = pd.Series(pca.explained_variance_ratio_, index=[f"PC{i+1}" for i in range(ncomp)])
loadings = pd.DataFrame(pca.components_, columns=rets_std.columns, index=evr.index)

print("\n=== PCA explained variance ratio ===")
print(evr.round(4))

print("\n=== PCA loadings (first 3 PCs, first 12 tickers) ===")
print(loadings.iloc[:3, :12].round(3))


=== PCA explained variance ratio ===
PC1     0.6013
PC2     0.1324
PC3     0.0414
PC4     0.0383
PC5     0.0195
PC6     0.0161
PC7     0.0140
PC8     0.0127
PC9     0.0114
PC10    0.0108
dtype: float64

=== PCA loadings (first 3 PCs, first 12 tickers) ===
Ticker    NEE     SO    CEG    DUK    VST    AEP    SRE      D    XEL    EXC  \
PC1     0.138  0.208  0.066  0.199  0.044  0.204  0.181  0.193  0.194  0.197   
PC2     0.055 -0.133  0.403 -0.177  0.433 -0.104  0.196  0.060 -0.028 -0.159   
PC3     0.388 -0.070 -0.266 -0.070 -0.225 -0.094  0.105  0.153  0.085 -0.052   

Ticker    PEG    ETR  
PC1     0.166  0.195  
PC2     0.227  0.127  
PC3    -0.184 -0.175  


In [142]:
X = rets_std.values  # shape T x N
U, s, Vt = np.linalg.svd(X, full_matrices=False)

print("\n=== SVD singular values (first 10) ===")
print(np.round(s[:10], 6))

print("\n=== Right singular vectors V^T (first 3 rows, first 12 tickers) ===")
print(pd.DataFrame(Vt[:3, :12], columns=rets_std.columns[:12], index=["SV1","SV2","SV3"]).round(3))



=== SVD singular values (first 10) ===
[48.075748 22.562936 12.608913 12.13472   8.658493  7.862717  7.341895
  6.975482  6.608176  6.454468]

=== Right singular vectors V^T (first 3 rows, first 12 tickers) ===
Ticker    NEE     SO    CEG    DUK    VST    AEP    SRE      D    XEL    EXC  \
SV1    -0.138 -0.208 -0.066 -0.199 -0.044 -0.204 -0.181 -0.193 -0.194 -0.197   
SV2     0.055 -0.133  0.403 -0.177  0.433 -0.104  0.196  0.060 -0.028 -0.159   
SV3     0.388 -0.070 -0.266 -0.070 -0.225 -0.094  0.105  0.153  0.085 -0.052   

Ticker    PEG    ETR  
SV1    -0.166 -0.195  
SV2     0.227  0.127  
SV3    -0.184 -0.175  
